# 패키지 임포트

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
import os
import string
import re
import json
import keras
from data_preprocessing import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 데이터 로드

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/bilm-tf-master/textdataset/WELFake/WELFake_Dataset.csv')

# 전처리

In [3]:
df = df.dropna()
df.isnull().sum()

df.drop(columns=['Unnamed: 0'],inplace=True)

df.drop_duplicates(inplace=True)

X = df.drop(columns=['label'])
y = df['label']

X[['title','text']] = X[['title','text']].applymap(lambda x:remove_punctuation(x))
X[['title','text']] = X[['title','text']].applymap(lambda x:x.lower())

X=X['title']+' '+X['text']

X = X.apply(clean_text)
X = list(X)

pattern = '[^a-z ]'
Clean_X=[]
for sen in X:
    Clean_X.append(re.sub(pattern, '', str(sen)))

clean_df = pd.DataFrame({'Clean_X': Clean_X, 'y': y})

fake_df = clean_df[clean_df['y'] == 0]
real_df = clean_df[clean_df['y'] == 1]

fake_x=list(fake_df['Clean_X'])
real_x=list(real_df['Clean_X'])

real_selected_lst = []
fake_selected_lst = []

for sen in real_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        real_selected_lst.append(sen)
        
for sen in fake_x:
    word_count = len(sen.split())
    if 10 <= word_count < 2000:
        fake_selected_lst.append(sen)
        
X=real_selected_lst[:10000]+fake_selected_lst[:10000]
y=[0]*10000+[1]*10000

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
to_txt=x_train+x_test

In [4]:
y=y_train+y_test

In [5]:
encoder=LabelEncoder()

encoder.fit(y)

label=encoder.transform(y)

y_train=list(label[:16000])
y_test=list(label[16000:])

In [6]:
#불용어 불러오기
with open('C:/Users/user/Desktop/english.txt', 'r', encoding='utf-8') as file:
    stopwords = [line.strip() for line in file]

In [7]:
vect = CountVectorizer(stop_words=stopwords)
X_dtm = vect.fit_transform(to_txt)
X_dtm = X_dtm.toarray()
X_new = SelectKBest(chi2, k=10000).fit(X_dtm, y)
TorF = X_new.get_support()
word_view=np.array(vect.get_feature_names_out())
feature_lst10000=word_view[TorF]

C:\ProgramData\anaconda3\envs\class\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'daren', 'hadn', 'herse', 'himse', 'itse', 'mayn', 'mightn', 'mon', 'mustn', 'myse', 'needn', 'oughtn', 'shan'] not in stop_words.
  warnings.warn(


# tensor space model 생성

In [24]:
doc_lst=[]
for sen in x_train:
    doc_lst.append(sen.split())

In [25]:
test_lst=[]
for sen in x_test:
    test_lst.append(sen.split())

In [ ]:
#Train_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(16000):
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Train_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Train_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

train_max_feature=max(count_lst)

In [ ]:
#Test_data에서 문서가 갖고 있는 선별한 feauture의 수 확인
count_lst=[]
for i in range(4000):
    
    total_feature_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            total_feature_cnt+=1
    count_lst.append(total_feature_cnt)
    
print('Test_data에서 가장 많은 feature를 가진 문서의 경우 feature',max(count_lst),' 개를 가짐')
print('Test_data에서 가장 적은 feature를 가진 문서의 경우 feature',min(count_lst),' 개를 가짐')

test_max_feature=max(count_lst)

max_feature=max(train_max_feature,test_max_feature)

In [26]:
#1-Channel TextCuboid 생성
textcuboid=[]
for i in range(16000):
    frame1=np.zeros((max_feature,256))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in doc_lst[i]:
            #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
            frame1[idx_cnt]=np.load('./elmo_embedding/train(WELFake256)/doc%d.npy'%i)[doc_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid.append(frame1)

textcuboid=np.array(textcuboid)

np.save('./1-Channel textcuboid_WELFake(elmo).npy',textcuboid)

In [27]:
#1-Channel TextCuboid 생성
textcuboid_test=[]

for i in range(4000):
    frame1=np.zeros((max_feature,256))
    idx_cnt=0
    for j in range(10000):
        if feature_lst10000[j] in test_lst[i]:
            #문서에서 선별한 단어(feature)의 위치를 찾아 임베딩 벡터 추출
            frame1[idx_cnt]=np.load('./elmo_embedding/test(WELFake256)/test%d.npy'%i)[test_lst[i].index(feature_lst10000[j])]
            idx_cnt+=1
    textcuboid_test.append(frame1)

textcuboid_test=np.array(textcuboid_test)
np.save('./1-Channel textcuboid_test_WELFake(elmo).npy',textcuboid_test)